In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
from llama_model_utils import LlamaAttention,LlamaSdpaAttention,LlamaMLP,LlamaDecoderLayer,LlamaModel,LlamaForCausalLM

(Re-)Loading modeling...


In [3]:
import torch
# from transformers.modeling_llama import LlamaForCausalLM
import transformers
# from transformers import AutoTokenizer
from datasets import load_dataset

In [4]:
torch.nn.Linear.reset_parameters = lambda x: None
# model = LlamaForCausalLM.from_pretrained('../llama-2-13b/', torch_dtype=torch.bfloat16)

In [5]:
local_model_path = "facebook/layerskip-llama2-7B"#: str = args.model

# import modeling_llama
# from modeling_llama import LlamaForCausalLM
# initialize model
tokenizer = transformers.AutoTokenizer.from_pretrained(local_model_path)
model = transformers.AutoModelForCausalLM.from_pretrained(
    local_model_path,
    use_safetensors=True,
    device_map="auto",
    torch_dtype=torch.float16,
)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
model = LlamaForCausalLM.from_pretrained(local_model_path, torch_dtype=torch.bfloat16)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
model = model.to('cuda:0').eval()

In [234]:
# tokenizer = AutoTokenizer.from_pretrained('../llama-2-13b/')

In [8]:
prompts = []

In [9]:
import os
os.environ['CURL_CA_BUNDLE'] = ''
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# xsum = load_dataset('xsum').shuffle(4242)
cnn = load_dataset('cnn_dailymail', '3.0.0').shuffle(4242)
# cnn = load_dataset('cnn_dm_summarization', '3.0.0').shuffle(4242)

Using the latest cached version of the dataset since cnn_dailymail couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration '3.0.0' at /home/tiger/.cache/huggingface/datasets/cnn_dailymail/3.0.0/0.0.0/96df5e686bee6baa90b8bee7c28b81fa3fa6223d (last modified on Mon Dec  9 19:50:25 2024).


In [10]:
for i in range(4):
    item = cnn['train'][i+100]
    cnn_context = 'Article: ' + item['article'] + '\nSummary: ' + item['highlights'].replace('\n', '')
    
    item = cnn['train'][i]
    prompt = cnn_context + '\nArticle: ' + item['article'] + '\nSummary:'
    prompts.append(prompt)

In [ ]:
# for i in range(4):
#     item = xsum['train'][i+100]
#     xsum_context = 'Article: ' + item['document'] + '\nSummary: ' + item['summary'].replace('\n', '')
    
#     item = xsum['train'][i]
#     prompt = xsum_context + '\nArticle: ' + item['document'] + '\nSummary:'
#     prompts.append(prompt)
     

In [11]:
from searching import LayerSkippingSearching

layer_searching = LayerSkippingSearching(model, tokenizer, prompts, evaluate_config={"generate_fn": "essg", "max_new_tokens": 32})

In [11]:
# attn_skip_layers = [8,10,15,18,20,24,25,26,27,28,29,30,31]
# mlp_skip_layers = [8,10,15,18,20,24,25,26,27,28,29,30,31]

# params = {f"x{i}": 0.0 for i in range(layer_searching.config.num_hidden_layers * 2)}
# for i in attn_skip_layers:
#     params[f"x{i}"] = 1.0
# for i in mlp_skip_layers:
#     params[f"x{i+32}"] = 1.0

# layer_searching._black_box_evaluate_function(params = params)

In [12]:
layer_searching.probe([8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31], [8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31])
# layer_searching.probe([3, 5, 6, 8, 10, 11, 14, 15, 18, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31], [6, 9, 10, 11, 15, 24, 25, 27, 28])

In [13]:
layer_searching.search(300)

|   iter    |  target   |    x0     |    x1     |    x10    |    x11    |    x12    |    x13    |    x14    |    x15    |    x16    |    x17    |    x18    |    x19    |    x2     |    x20    |    x21    |    x22    |    x23    |    x24    |    x25    |    x26    |    x27    |    x28    |    x29    |    x3     |    x30    |    x31    |    x32    |    x33    |    x34    |    x35    |    x36    |    x37    |    x38    |    x39    |    x4     |    x40    |    x41    |    x42    |    x43    |    x44    |    x45    |    x46    |    x47    |    x48    |    x49    |    x5     |    x50    |    x51    |    x52    |    x53    |    x54    |    x55    |    x56    |    x57    |    x58    |    x59    |    x6     |    x60    |    x61    |    x62    |    x63    |    x7     |    x8     |    x9     |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

([0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31],
 [0,
  1,
  3,
  4,
  5,
  6,
  7,
  9,
  11,
  12,
  13,
  14,
  15,
  17,
  18,
  19,
  20,
  23,
  24,
  25,
  26,
  27])

In [14]:
layer_searching.get_solution()

([0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31],
 [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  9,
  10,
  11,
  13,
  14,
  15,
  16,
  17,
  18,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  30])

In [20]:
layer_searching.get_solution()

([2, 3, 9, 10, 11, 12, 13, 17, 19, 20, 21, 23, 25, 27, 28, 30],
 [3, 6, 7, 9, 19, 20, 21, 22, 28, 31])

In [15]:
layer_searching.get_solution()

([8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31],
 [8,
  9,
  10,
  11,
  12,
  13,
  15,
  16,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31])

In [ ]:
def forward(
    model: transformers.LlamaForCausalLM,
    input_ids: torch.Tensor,
    past_key_values: Optional[List[Tuple[torch.Tensor, torch.Tensor]]],
) -> ForwardResult:
    device = input_ids.device
    batch_size, seq_length = input_ids.shape

    seq_length_with_past = seq_length
    past_key_values_length = 0

    if past_key_values is not None:
        past_key_values_length = past_key_values[0][0].shape[2]
        seq_length_with_past = seq_length_with_past + past_key_values_length
    past_key_values = transformers.cache_utils.DynamicCache.from_legacy_cache(past_key_values)

    position_ids = torch.arange(
        past_key_values_length,
        seq_length + past_key_values_length,
        dtype=torch.long,
        device=device,
    )
    position_ids = position_ids.unsqueeze(0).view(-1, seq_length)
    attention_mask = input_ids.new_ones(
        (batch_size, seq_length_with_past),
        dtype=torch.bool,
    )
    inputs_embeds = model.model.embed_tokens(input_ids)
    attention_mask = _prepare_decoder_attention_mask(
        model,
        attention_mask,
        (batch_size, seq_length),
        inputs_embeds,
        past_key_values_length,
    )

    hidden_states = inputs_embeds
    for decoder_layer in model.model.layers:
        hidden_states, past_key_values = decoder_layer(
            hidden_states,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_value=past_key_values,
            output_attentions=False,
            use_cache=True,
            padding_mask=None,
        )

    past_key_values = past_key_values.to_legacy_cache()
    hidden_states = model.model.norm(hidden_states)
    logits = model.lm_head(hidden_states)

    return ForwardResult(
        logits=logits, past_key_values=past_key_values
    )


# TODO: update forward_early(...) to use transformers' new KV cache implementation rather than legacy.
def forward_early(
    model: transformers.LlamaForCausalLM,
    input_ids: torch.Tensor,
    past_key_values: Optional[List[Tuple[torch.Tensor, torch.Tensor]]],
    exit_layer: int,
    exit_query_cache: Optional[List[torch.Tensor]],
) -> ForwardResult:
    device = input_ids.device
    batch_size, seq_length = input_ids.shape

    seq_length_with_past = seq_length
    past_key_values_length = 0

    if past_key_values is not None:
        past_key_values_length = past_key_values[0][0].shape[2]
        seq_length_with_past = seq_length_with_past + past_key_values_length
    past_key_values = transformers.cache_utils.DynamicCache.from_legacy_cache(past_key_values)

    position_ids = torch.arange(
        past_key_values_length,
        seq_length + past_key_values_length,
        dtype=torch.long,
        device=device,
    )
    position_ids = position_ids.unsqueeze(0).view(-1, seq_length)
    attention_mask = input_ids.new_ones(
        (batch_size, seq_length_with_past),
        dtype=torch.bool,
    )
    inputs_embeds = model.model.embed_tokens(input_ids)
    attention_mask = _prepare_decoder_attention_mask(
        model,
        attention_mask,
        (batch_size, seq_length),
        inputs_embeds,
        past_key_values_length,
    )

    hidden_states = inputs_embeds
    for decoder_layer in model.model.layers[:exit_layer]:
        hidden_states, past_key_values = decoder_layer(
            hidden_states,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_value=past_key_values,
            output_attentions=False,
            use_cache=True,
            padding_mask=None,
        )

    past_key_values = past_key_values.to_legacy_cache()

    # next_cache = next_decoder_cache
    if exit_query_cache is None:
        exit_query_cache = hidden_states
    else:
        exit_query_cache = torch.cat([exit_query_cache, hidden_states], dim=1)

    hidden_states = model.model.norm(hidden_states)

    logits = model.lm_head(hidden_states)
    return ForwardResult(
        logits=logits, past_key_values=past_key_values, exit_query_cache=exit_query_cache
    )
